In [6]:
import pandas as pd
import datasets

In [7]:
df_text = pd.read_csv("data/data_sample_20251111.csv", encoding = "cp949")
df_text.head()

,subject_id,first_admit,last_discharge,deathtime,survival_time,death_event,system,question,text,assistant
0,14711614,2169-08-25 3:39,2169-08-25 22:03,NaN,0.766667,0,You are the world’s leading expert in survival...,Please summarize the following discharge summa...,\nName: ___ Unit No: __...,A female patient presented with vertigo; on ph...
1,17622322,2144-11-14 8:45,2149-06-18 17:10,NaN,1677.350694,0,You are the world’s leading expert in survival...,Please summarize the following discharge summa...,\nName: ___ Unit No: ___\...,A male patient presented with right knee pain;...
2,10063762,2125-04-24 20:43,2125-04-25 15:37,NaN,0.787500,0,You are the world’s leading expert in survival...,Please summarize the following discharge summa...,\nName: ___ Unit No: ___\n ...,A female patient presented with dizziness; on ...
3,15732392,2133-09-22 20:12,2134-10-20 18:25,NaN,392.925694,0,You are the world’s leading expert in survival...,Please summarize the following discharge summa...,\r\nName: ___ Unit No: _...,A male patient presented with hematemesis and ...
4,10172264,2116-05-08 18:41,2117-05-29 17:59,NaN,385.970833,0,You are the world’s leading expert in survival...,Please summarize the following discharge summa...,\nName: ___ Unit No: ___\n...,A female patient presented with right calf pai...


In [24]:
df_text.text.str.len().max()

np.int64(24824)

In [25]:
df_text.assistant.str.len().max()

np.int64(559)

In [60]:
"You are the world’s leading expert in survival analysis.\
 From a discharge summary, extract Chief Complaint, Physical Exam, and Admission Labs and produce one sentence.\
 The sentence will be used for hazard calculation, so be precise, clinically accurate, and concise."

'You are the world’s leading expert in survival analysis. From a discharge summary, extract Chief Complaint, Physical Exam, and Admission Labs and produce one sentence. The sentence will be used for hazard calculation, so be precise, clinically accurate, and concise.'

In [ ]:
"Please summarize the following discharge summary\
 in one sentence focusing on Chief Complaint, Physical Exam, and Admission Labs."

'Please summarize the following discharge summary in one sentence focusing on Chief Complaint, Physical Exam, and Admission Labs.'

In [50]:
system_prompt = df_text.system[0]
ds = datasets.Dataset.from_pandas(df_text)
columns_to_remove = [f for f in list(ds.features) if f != "subject_id"]

In [52]:
train_ds = ds.map(
    lambda sample:
    {"messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["question"] + "\n" + sample["text"]},
        {"role": "assistant", "content": sample["assistant"]}
    ]}
)

Map: 100%|██████████| 100/100 [00:00<00:00, 2100.27 examples/s]


In [53]:
train_ds = train_ds.map(remove_columns = columns_to_remove, batched = False)
train_ds = train_ds.train_test_split(test_size = 0.1, seed = 42)

Map: 100%|██████████| 100/100 [00:00<00:00, 4171.11 examples/s]


In [ ]:
train_ds["train"].to_json("data/sft_train_dataset.json", orient = "records", force_ascii = False)
train_ds["test"].to_json("data/sft_test_dataset.json", orient = "records", force_ascii = False)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 273.99ba/s]


108594

In [4]:
train_ds = datasets.load_dataset("json", data_files = "data/sft_train_dataset.json", split = "train")